In [1]:
# Import libraries
import numpy as np
import pandas as pd
import scipy
# from sklearn.linear_model import ElasticNet, Ridge, Lasso, LinearRegression
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.neighbors import KNeighborsRegressor
# from sklearn.ensemble import RandomForestRegressor
from xgboost.sklearn import XGBRegressor
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, DotProduct, Matern, RationalQuadratic, WhiteKernel
from sklearn.model_selection import GridSearchCV, KFold
# from sklearn.preprocessing import StandardScaler
import pickle

In [2]:
# CHANGE FILENAME HERE
test_dataset_file = "sample_dataset.csv"
# SELECT OPTION: 'full' or 'partial'
TRAINING_OPTION = 'partial' 

In [3]:
# Read .csv file
Dataset = pd.read_csv(test_dataset_file)
# Constants
TARGETS = ['Water_Absorption_%','Hardness','Thermal_Conductivity_(mW/m.K)']
NR_FEATURES = ['Clarifier_1','Clarifier_2','Clarifier_3',
                'Polymer_3','UV_absorber_1','UV_absorber_2',
                'Filler_2','Filler_3']
METRICS_NAME = ['r2', 'MAE', 'RMSE']
METRICS = [r2_score, mean_absolute_error, mean_squared_error]
# Load models and standardization parameters
params_filename = 'standardizer_'+TRAINING_OPTION+'_dataset.dump'
models_filename = TRAINING_OPTION+'_dataset_models.dump'
mu, sigma = pickle.load(open(params_filename,"rb"))
models = pickle.load(open(models_filename,"rb"))
# PREPROCESSING
# Select only features from dataset
X_test = Dataset.drop(columns=['name']+TARGETS)
# add entropy as a feature
X_test['entropy']=scipy.stats.entropy(X_test, axis=1)
# removing some features may help?
if TRAINING_OPTION == 'partial':
    X_test = X_test.drop(columns=NR_FEATURES)
# Select only targets from dataset
Y_test = Dataset[TARGETS]
# Standardization
X_test = (X_test - mu)/sigma
# y_train = Y_train[TARGETS[0]]

[00:30:01] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [19]:
# Predictions evaluation 
for k,target in enumerate(TARGETS):
    y_test = Y_test[target]
    y_predicted = models[k].predict(X_test)
    print('TARGET:',target) 
    print('model:',str(models[k]).split('(')[0])
    
    acc = [print(name,metric(y_test, y_predicted)) for metric,name in zip(METRICS,METRICS_NAME)]

TARGET: Water_Absorption_%
model: GaussianProcessRegressor
r2 0.988767815639689
MAE 0.043564602746678005
RMSE 0.0027052364150465936
TARGET: Hardness
model: GaussianProcessRegressor
r2 0.9986375263615909
MAE 0.6614992086023443
RMSE 0.7914840345120293
TARGET: Thermal_Conductivity_(mW/m.K)
model: XGBRegressor
r2 0.7178920515576901
MAE 5.54810733645758
RMSE 44.86138626958788
